In [1]:
import math
import numpy as np
from numpy import linalg as LA

In [2]:
EPS_ZEIDEL = 1e-12
ITERATIONS_ZEIDEL = 200000

In [3]:
def equalVector(a, b, eps):
    return math.sqrt(sum([(a[i] - b[i])**2 for i in range(len(a))])) < eps

In [4]:
def getLambdas(A):
    m = LA.eigvals(A)
    maxV = np.max(m)
    minV = np.min(m)
    return (maxV, minV)

In [5]:
def zeidelMethod(A, b):
    n = len(A)
    x = [100.0 for i in range(n)]
    xn = [.0 for i in range(n)]
    gotNAN = False
    curIterations = 0
    (ma, mi) = getLambdas(A)
    t = 1
    if (mi > 0):
        t = 2 / (ma + mi)
        A = t * A
        b = t * b
    if (mi <= 0 and ma > 0):
        t = 2 / (ma + 1)
        A = t * A
        b = t * b
    print(f"Zeidel correction coefficient = {t}")
    eA = np.eye(n) - A
    B2 = np.zeros((n, n))
    for i in range (n-1):
        for j in range(i + 1, n):
            B2[i][j] = A[i][j]
    b2Norm = LA.norm(B2, 2)
    bNorm = LA.norm(eA, 2)
    coef = (1 - bNorm)/b2Norm
    print(f"||B2|| is {b2Norm}")
    print(f"||B|| is {bNorm}")
    print(f"Поправочный коэффициент = (1 - ||B||)/(||B2||) = {coef}")
    while (not equalVector(x, xn, coef*EPS_ZEIDEL)) and (curIterations < ITERATIONS_ZEIDEL) and (not gotNAN):
        x = np.copy(xn)
        curIterations += 1
        for i in range(n):
            s1 = - sum(A[i][j] * xn[j] for j in range(i)) / A[i][i]
            s2 = - sum(A[i][j] * x[j] for j in range(i + 1, n)) / A[i][i]
            xn[i] = (b[i] / A[i][i] + s1 + s2)
            if np.isnan(xn[i]):
                print("Zeidel Got nan")
                gotNAN = True
                xn = [np.Inf for i in range(n)]
                print(f"Zeidel Iterations = {ITERATIONS_ZEIDEL}. Max is {ITERATIONS_ZEIDEL} iterations.")
                return xn
                
    print(f"Zeidel Iterations = {curIterations}. Max is {ITERATIONS_ZEIDEL} iterations.")
    return xn

In [6]:
def zeidelMethodRelax(A, b):
    n = len(A)
    x = [100.0 for i in range(n)]
    xn = [.0 for i in range(n)]
    Relax = 1.6
    gotNAN = False
    curIterations = 0
    (ma, mi) = getLambdas(A)
    t = 1
    if (mi != 0):
        t = 2 / (ma + mi)
        A = t * A
        b = t * b
    if (mi <= 0 and ma > 0):
        t = 2 / (ma + 1)
        A = t * A
        b = t * b
    print(f"Zeidel correction coefficient = {t}")
    eA = np.eye(n) - A
    B2 = np.zeros((n, n))
    for i in range (n-1):
        for j in range(i + 1, n):
            B2[i][j] = A[i][j]
    b2Norm = LA.norm(B2, 2)
    bNorm = LA.norm(eA, 2)
    coef = (1 - bNorm)/b2Norm
    print(f"||B2|| is {b2Norm}")
    print(f"||B|| is {bNorm}")
    print(f"Поправочный коэффициент = (1 - ||B||)/(||B2||) = {coef}")
    while (not equalVector(x, xn, coef * EPS_ZEIDEL)) and (curIterations < ITERATIONS_ZEIDEL) and (not gotNAN):
        x = np.copy(xn)
        curIterations += 1
        for i in range(n):
            s1 = - sum(A[i][j] * xn[j] for j in range(i)) / A[i][i]
            s2 = - sum(A[i][j] * x[j] for j in range(i + 1, n)) / A[i][i]
            xn[i] = (b[i] / A[i][i] + s1 + s2)
            xn[i] = Relax * xn[i] + (1 - Relax) * x[i]
            if np.isnan(xn[i]):
                print("Zeidel Relax Got nan")
                gotNAN = True
                xn = [np.Inf for i in range(n)]
                print(f"Zeidel Iterations = {ITERATIONS_ZEIDEL}. Max is {ITERATIONS_ZEIDEL} iterations.")
                return xn
            
    print(f"Zeidel Relax Iterations = {curIterations}. Max is {ITERATIONS_ZEIDEL} iterations.")
    return xn

Критерии остановки метода Зейделя - по близости двух последовательных приближений и ограничения максимального числа итераций (2000000). Вводится поправочный коэффициент $$\frac{1 - ||B||}{||B_{2}||}$$. 

Вектор начального приближения заполняем нулями. Коэффициент релаксации - 1.6, так как именно такой коэффициент, исходя из экспериментов, ускоряет медленную сходимость, как в случае с матрицей Гильберта. Также мы пробовали коэффициент нижней релаксации (< 1), которая может дать сходимость, если ее нет, но пришли к выводу, что наиболее оптимальный результат получается при верхней релаксации.
Использовались меры 2го порядка(спектральные)

Метод Зайделя ведет себя хорошо на симметричных положительно определенных матрицах (все собственные числа положительные). При генерации рандомной матрицы мы сделали так, чтобы все ее собственные числа были вещественными (сложили A.T и A, где A случайная матрица), но при этом эти числа могут быть отрицательными, тогда при подсчете коэффициента $$\tau$$ $${\lambda_{min}} < 0$$, и, возможно, и $${\lambda_{max}} < 0$$. В таком случае непонятно, каким образом находить оптимальное $$\tau$$, поэтому оно останется 1.
Явление положительной определенности довольно редкое, очень большая вероятность, что рандомная матрица не обладает нужными свойствами. Поэтому метод чаще всего не сходится на рандомных матрицах